## 1. Import libraries

In [1]:
import os
import urllib.request
from types import SimpleNamespace
from urllib.error import HTTPError

import lightning as L
import matplotlib
import matplotlib.pyplot as plt
import matplotlib_inline.backend_inline
import numpy as np
import seaborn as sns
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision

%matplotlib inline
from IPython.display import HTML, display
from lightning.pytorch.callbacks import LearningRateMonitor, ModelCheckpoint
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint
from PIL import Image
from torchvision import transforms
from torchvision.datasets import CIFAR10

matplotlib_inline.backend_inline.set_matplotlib_formats("svg", "pdf")  # For export
matplotlib.rcParams["lines.linewidth"] = 2.0
sns.reset_orig()

import pandas as pd
import torch
from torch import nn
import torchmetrics
from torchvision import transforms
from torch.utils.data import DataLoader, random_split
import pytorch_lightning as pl
from torchvision.datasets import FashionMNIST
from pytorch_lightning.loggers import WandbLogger

from torch.utils.data import DataLoader
import torchmetrics
import math
# PyTorch
# Torchvision

## 2. Define a dataset

In [2]:
import os
import pandas as pd
from torchvision.io import read_image
from torch.utils.data import Dataset
import torch
import glob
from torchvision import transforms


class DasDataset(Dataset):
    def __init__(self, file_dir='../data/interim/', transform=None, target_transform=None):
        self.transform = transform
        self.target_transform = target_transform
        # file paths
        self.file_paths = glob.glob(file_dir + "/**/*.hdf5", recursive = True)
        file_shapes = []
        max_val = 0
        min_val = 0
        for i in self.file_paths:
            df = pd.read_hdf(i)
            file_shapes.append(df.shape)
            min_i = df.min().min()
            max_i = df.max().max()
            if  min_i < min_val:
                min_val = min_i
            if max_i > max_val:
                max_val = max_i

        self.max_val = max_val
        self.min_val = min_val

        # keep only files that have correct size (63, 50)
        correct_idx = [index for index, value in enumerate(file_shapes) if value == (63, 50)]
        self.file_paths = [self.file_paths[index] for index in correct_idx]

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        file_path = self.file_paths[idx]
        image = pd.read_hdf(file_path).values.astype(np.float32)
        image = (image - self.min_val) / (self.max_val - self.min_val)
        image = torch.tensor(image).unsqueeze(0)
        label = file_path.split('/')[3]
        if label == 'rock':
            label = 1
        elif label == 'basketball':
            label = 0
        label = torch.tensor(label)
        # if self.transform:
        #     image = self.transform(image)
        # if self.target_transform:
        #     label = self.target_transform(label)
        return image, label

In [3]:
data = DasDataset(transform=None, target_transform=None)
torch.manual_seed(2252)
train_set, val_set, test_set = torch.utils.data.random_split(data, [100, 50, 71])

train_loader = DataLoader(train_set, batch_size=4, shuffle=False)
val_loader = DataLoader(val_set, batch_size=4, shuffle=False)
test_loader = DataLoader(test_set, batch_size=4, shuffle=False)

In [4]:
# checking data
print(len(train_set), len(val_set), len(test_set))

100 50 71


In [5]:
# checking data
labels = [i[1].item() for i in data]
pd.Series(labels).value_counts()

1    117
0    104
dtype: int64

## 3. Define dataset by Lighning Data Module

In [6]:
import lightning.pytorch as pl
from torch.utils.data import random_split, DataLoader

# Note - you must have torchvision installed for this example
from torchvision.datasets import MNIST
from torchvision import transforms


class DasDataModule(pl.LightningDataModule):
    def __init__(self):
        super().__init__()
        # self.data_dir = data_dir
        # self.transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])

    def train_dataloader(self):
        return DataLoader(train_set, batch_size=4)

    def val_dataloader(self):
        return DataLoader(val_set, batch_size=4)

    def test_dataloader(self):
        return DataLoader(test_set, batch_size=4)

## 3. Define model

In [7]:
# import the necessary packages
from torch.nn import Module
from torch.nn import Conv2d
from torch.nn import Linear
from torch.nn import MaxPool2d
from torch.nn import ReLU
from torch.nn import LogSoftmax
from torch import flatten

## Resnet

In [8]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchmetrics
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import pytorch_lightning as pl

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# ResNet model
class ResNetModel(pl.LightningModule):
    def __init__(self, num_classes=1):
        super().__init__()
        self.resnet = torch.hub.load('pytorch/vision:v0.10.0', 'resnet152', pretrained=True)
        self.resnet.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        in_features = self.resnet.fc.in_features
        self.resnet.fc = nn.Linear(in_features, num_classes)
        self.loss_fn = nn.BCEWithLogitsLoss()
        self.accuracy = torchmetrics.Accuracy(task='binary')

    def forward(self, x):
        return self.resnet(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = self.loss_fn(logits.squeeze(), y.float())
        self.log('train_loss', loss)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = self.loss_fn(logits.squeeze(), y.float())
        acc = self.accuracy(torch.sigmoid(logits).squeeze(), y.float())
        self.log('val_loss', loss, prog_bar=True)
        self.log('val_acc', acc, prog_bar=True)

    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=1e-3)
    
    def predict_step(self, batch, batch_idx, dataloader_idx=0):
        return self(batch)

# Set up data directories and instantiate DataModule and ResNetModel
data_dir = "path/to/your/data/folders"
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)),
])
resnet_model = ResNetModel()


checkpoint_callback = pl.callbacks.ModelCheckpoint(
    monitor='val_acc',
    dirpath='./',
    filename='models-{epoch:02d}-{val_acc:.2f}',
    save_top_k=1,
    mode='max'
)

# Train the model using Lightning Trainer
trainer = pl.Trainer(max_epochs=100, callbacks=[checkpoint_callback], accelerator='cpu') # , gpus=1 if torch.cuda.is_available() else 0
trainer.fit(model=resnet_model, train_dataloaders=train_loader, val_dataloaders=val_loader)

Using cache found in /Users/kptruong/.cache/torch/hub/pytorch_vision_v0.10.0
/Users/kptruong/miniconda3/envs/das-env/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/kptruong/miniconda3/envs/das-env/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet152_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet152_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/kptruong/miniconda3/envs/das-env/lib/python3.11/site-packages/pytorch_lightning/t

Sanity Checking DataLoader 0:  50%|█████     | 1/2 [00:00<00:00, 12.36it/s]

/Users/kptruong/miniconda3/envs/das-env/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:432: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


/Users/kptruong/miniconda3/envs/das-env/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:432: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/Users/kptruong/miniconda3/envs/das-env/lib/python3.11/site-packages/pytorch_lightning/loops/fit_loop.py:280: PossibleUserWarning: The number of training batches (25) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Epoch 99: 100%|██████████| 25/25 [00:07<00:00,  3.16it/s, v_num=53, val_loss=1.160, val_acc=0.600]

`Trainer.fit` stopped: `max_epochs=100` reached.


Epoch 99: 100%|██████████| 25/25 [00:07<00:00,  3.16it/s, v_num=53, val_loss=1.160, val_acc=0.600]


In [9]:
BEST_MODEL_PATH = "/Users/kptruong/NTNU/dasly/notebooks/models-epoch=77-val_acc=0.96.ckpt" #checkpoint_callback.best_model_path
pretrained_model = ResNetModel().load_from_checkpoint(BEST_MODEL_PATH)
pretrained_model.eval()
pretrained_model.freeze()

Using cache found in /Users/kptruong/.cache/torch/hub/pytorch_vision_v0.10.0
/Users/kptruong/miniconda3/envs/das-env/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/kptruong/miniconda3/envs/das-env/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet152_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet152_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Using cache found in /Users/kptruong/.cache/torch/hub/pytorch_vision_v0.10.0


In [13]:
# data = CustomImageDataset()
# train_set, val_set, test_set = torch.utils.data.random_split(data, [100, 50, 71])

# train_loader = DataLoader(train_set, batch_size=1, shuffle=True)
# val_loader = DataLoader(val_set, batch_size=1, shuffle=True)
# test_loader = DataLoader(test_set, batch_size=1, shuffle=True)

In [14]:
# Load the trained model
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

loader = test_loader

predictions = []
with torch.no_grad():
    for inputs, labels in loader:  # Assuming test_loader returns (input, label) pairs, and we only care about inputs here
        # print(labels)
        inputs, labels = inputs.to(device), labels.to(device)
        inputs = inputs.to(device)  # Move inputs to the same device as the model
        # inputs = torch.unsqueeze(inputs, 0)  # Add a batch dimension to the input data
        outputs = pretrained_model(inputs)
        predicted = torch.greater(outputs, 0.5)
        predictions.extend(predicted.cpu().numpy())  # Append predictions to the list

predictions = [i[0].astype(int) for i in predictions]
print(len(predictions))

71


In [16]:
true_labels = [i[1].item() for i in loader.dataset]
acc = sum([i == j for i, j in zip(predictions, true_labels)]) / len(true_labels)
print(f'{acc:.1%}')

91.5%
